# Embryo Metadata Annotation Tutorial for Biologists

This tutorial will guide you through using the embryo metadata annotation system to add biological annotations to your SAM2 segmentation results.

## What This System Does

- **Takes your SAM2 results** and lets you add biological information
- **Tracks phenotypes** (what you observe) at the frame level
- **Records genotypes** (genetic background) at the embryo level
- **Documents treatments** applied to embryos
- **Validates data** to prevent common mistakes
- **Safely handles** the special case of embryo death

## Basic Setup

First, let's load the system and initialize with your SAM2 data:

In [ ]:
# Import the annotation system
import sys
sys.path.append('scripts')

from annotations.embryo_metadata import EmbryoMetadata

# Load your SAM2 data
# Replace 'your_sam2_file.json' with your actual SAM2 file
metadata = EmbryoMetadata('path/to/your_sam2_file.json')

print(f"Loaded {len(metadata.list_embryos())} embryos")
print(f"Sample embryos: {metadata.list_embryos()[:3]}")

## Adding Phenotype Annotations

Phenotypes are what you observe about the embryos (normal, edema, dead, etc.).

### Available Phenotypes
- **NORMAL**: Typical development
- **EDEMA**: Fluid accumulation/swelling
- **DEAD**: Embryo has died
- **CONVERGENCE_EXTENSION**: Defects in body axis extension
- **BLUR**: Motion blur in imaging
- **CORRUPT**: Imaging artifacts

In [ ]:
# Example: Add phenotype to all frames of an embryo
result = metadata.add_phenotype(
    phenotype="NORMAL",
    author="your_name",
    embryo_id="20240418_A01_e01",  # Replace with actual embryo ID
    target="all"  # Apply to all frames
)

print(f"Applied NORMAL to {result['count']} frames")
print(f"Frames: {result['applied_to'][:3]}...")  # Show first 3

In [ ]:
# Example: Add phenotype to specific frame range
result = metadata.add_phenotype(
    phenotype="EDEMA",
    author="your_name",
    embryo_id="20240418_A01_e01",
    target="100:200"  # Frames 100 to 199
)

print(f"Applied EDEMA to {result['count']} frames")

In [ ]:
# Example: Add phenotype to single frame
result = metadata.add_phenotype(
    phenotype="DEAD",
    author="your_name",
    embryo_id="20240418_A01_e01",
    target="250"  # Just frame 250
)

print(f"Marked embryo as DEAD at frame 250")

## Special Handling of Death

The system has special rules for the DEAD phenotype:

1. **Exclusivity**: DEAD cannot coexist with other phenotypes at the same frame
2. **Permanence**: Once dead, the embryo stays dead (all later frames)
3. **Safety**: If you try to add other phenotypes to dead frames, they're silently skipped

In [ ]:
# This will safely skip the dead frame (250) and only apply to living frames
result = metadata.add_phenotype(
    phenotype="BLUR",
    author="your_name",
    embryo_id="20240418_A01_e01",
    target="all"  # Applies to all frames except dead ones
)

print(f"Applied BLUR to {result['count']} frames")
if 'skipped_dead_frames' in result:
    print(f"Safely skipped {len(result['skipped_dead_frames'])} dead frames")

## Adding Genotype Information

Genotypes describe the genetic background of your embryos.

### Available Genes
- **WT**: Wild type
- **tmem67**: Transmembrane protein 67
- **lmx1b**: LIM homeobox transcription factor 1 beta
- **sox9a**: SRY-box transcription factor 9a
- **cep290**: Centrosomal protein 290
- **b9d2**: B9 domain containing 2
- **rpgrip1l**: RPGRIP1 like

### Available Zygosity
- **homozygous**: Two identical alleles
- **heterozygous**: Two different alleles
- **compound_heterozygous**: Two different mutant alleles
- **crispant**: CRISPR-generated mutation
- **morpholino**: Morpholino knockdown

In [ ]:
# Add genotype to an embryo
result = metadata.add_genotype(
    gene="tmem67",
    author="your_name",
    embryo_id="20240418_A01_e01",
    allele="sa1423",  # Optional: specific allele
    zygosity="homozygous"
)

print(f"Added genotype: {result['gene']} ({result['zygosity']})")

## Adding Treatment Information

Document any treatments applied to your embryos.

### Available Treatments
- **control**: No treatment
- **DMSO**: Dimethyl sulfoxide (vehicle)
- **PTU**: Propylthiouracil (pigment inhibitor)
- **BIO**: GSK-3 inhibitor
- **SB431542**: TGF-β inhibitor
- **DAPT**: Notch inhibitor
- **heat_shock**: Temperature stress
- **cold_shock**: Cold temperature stress

In [ ]:
# Add treatment information
result = metadata.add_treatment(
    treatment="PTU",
    author="your_name",
    embryo_id="20240418_A01_e01",
    temperature_celsius=28.5,
    concentration="200μM",
    notes="24-48hpf pigment inhibition"
)

print(f"Added treatment: {result['treatment']}")

## Viewing Your Annotations

Check what you've annotated so far:

In [ ]:
# Get summary for a specific embryo
summary = metadata.get_embryo_summary("20240418_A01_e01")

print(f"Embryo: {summary['embryo_id']}")
print(f"Experiment: {summary['experiment_id']}")
print(f"Total frames: {summary['total_snips']}")
print(f"Genotype: {summary['genotype']}")
print(f"Phenotype counts: {summary['phenotype_counts']}")
print(f"Treatments: {summary['treatment_count']}")

In [ ]:
# Get overall statistics
stats = metadata.get_stats()

print(f"Total embryos: {stats['embryo_count']}")
print(f"Total frames: {stats['total_snips']}")
print(f"Total phenotype annotations: {stats['total_phenotypes']}")
print(f"Embryos with genotypes: {stats['genotyped_embryos']}")

## Saving Your Work

**Important**: Always save your annotations when you're done!

In [ ]:
# Save all your annotations
metadata.save()

print("✅ Annotations saved successfully!")
print(f"Saved to: {metadata.annotations_path}")

## Common Workflows

### Workflow 1: Basic Phenotyping

1. Mark all embryos as NORMAL initially
2. Add specific phenotypes where observed
3. Mark death when it occurs
4. Save your work

In [ ]:
# Example workflow for multiple embryos
embryos_to_annotate = ["20240418_A01_e01", "20240418_A01_e02", "20240418_A01_e03"]

for embryo_id in embryos_to_annotate:
    # Start with normal
    metadata.add_phenotype("NORMAL", "your_name", embryo_id, "all")
    
    # Add specific observations
    # (You would adjust these based on your actual observations)
    if embryo_id.endswith("e02"):
        metadata.add_phenotype("EDEMA", "your_name", embryo_id, "150:200")
    elif embryo_id.endswith("e03"):
        metadata.add_phenotype("DEAD", "your_name", embryo_id, "180")

print("✅ Batch annotation complete")

### Workflow 2: Genotype Assignment

Add genetic background information to all your embryos:

In [ ]:
# Define your experimental groups
genotype_assignments = {
    "20240418_A01_e01": {"gene": "tmem67", "zygosity": "homozygous"},
    "20240418_A01_e02": {"gene": "tmem67", "zygosity": "heterozygous"},
    "20240418_A01_e03": {"gene": "WT", "zygosity": "homozygous"}
}

for embryo_id, genotype_info in genotype_assignments.items():
    metadata.add_genotype(
        gene=genotype_info["gene"],
        author="your_name",
        embryo_id=embryo_id,
        zygosity=genotype_info["zygosity"]
    )

print("✅ Genotype assignment complete")

## Error Handling

The system helps you avoid common mistakes:

In [ ]:
# This will show you helpful error messages
try:
    # Try to use an invalid phenotype
    metadata.add_phenotype("INVALID_PHENOTYPE", "your_name", "20240418_A01_e01")
except ValueError as e:
    print(f"❌ Error caught: {e}")
    print("The system shows you valid options when you make mistakes!")

## Tips for Success

1. **Save frequently** - Use `metadata.save()` after major annotation sessions
2. **Check your work** - Use `get_embryo_summary()` to review annotations
3. **Use ranges** - `target="100:200"` is efficient for annotating frame ranges
4. **Trust the DEAD logic** - The system safely handles death scenarios
5. **Read error messages** - They contain helpful solutions
6. **Be consistent** - Use the same author name throughout your session

## Getting Help

- Check error messages - they're designed to be helpful
- Use `help(metadata.add_phenotype)` to see function documentation
- Review this tutorial when you need a refresher

**Happy annotating! 🧬🔬**